# Dataset Join and Cleaning

Bu notebook quyidagilarni bajaradi:
1. CSV fayllarni tekshirish va validatsiya qilish
2. Ma'lumotlarni tozalash (currency, commas, categorical)
3. Barcha datasetlarni birlashtirish (join)
4. Final tozalangan datasetni saqlash


In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)

data_folder = Path('../converted_csv/')


✅ Kutubxonalar yuklandi!

📂 Ma'lumotlar papkasi: ..\converted_csv
   Papka mavjud: True


In [ ]:
# CSV fayllarni yuklash va validatsiya qilish
csv_files = {
    'application_metadata': 'application_metadata.csv',
    'demographics': 'demographics.csv',
    'credit_history': 'credit_history.csv',
    'loan_details': 'loan_details.csv',
    'financial_ratios': 'financial_ratios.csv',
    'geographic_data': 'geographic_data.csv'
}

datasets = {}

for name, filename in csv_files.items():
    filepath = data_folder / filename
    if not filepath.exists():
        continue
    
    try:
        df = pd.read_csv(filepath)
        datasets[name] = df
    except Exception as e:
        continue


📋 PHASE 1: CSV FAYLLARNI VALIDATSIYA QILISH

📄 APPLICATION_METADATA: application_metadata.csv
----------------------------------------------------------------------
✅ Fayl muvaffaqiyatli o'qildi
   O'lchami: (89999, 14)
   Qatorlar: 89,999
   Ustunlar: 14
   Ustunlar: ['customer_ref', 'application_id', 'application_hour', 'application_day_of_week', 'account_open_year', 'preferred_contact', 'referral_code', 'account_status_code', 'random_noise_1', 'num_login_sessions', 'num_customer_service_calls', 'has_mobile_app', 'paperless_billing', 'default']

   Ma'lumot turlari:
int64      10
object      3
float64     1
Name: count, dtype: int64

   ✅ Yo'qolgan qiymatlar yo'q

   🔑 Kalit ustun: customer_ref
      Noyob qiymatlar: 89,999
      Min: 10000, Max: 99998
      Takrorlanganlar: 0

   🎯 Target ustun: default
      Qiymatlar taqsimoti:
default
0    85405
1     4594
Name: count, dtype: int64
      Foizlar:
default
0    94.9
1     5.1
Name: proportion, dtype: float64

📄 DEMOGRAPHICS: demogr

In [ ]:
# Tozalash funksiyalari
def clean_currency(value):
    """Currency va vergullarni olib tashlash: '$17,700' → 17700.0"""
    if pd.isna(value):
        return np.nan
    
    # String ga o'tkazish
    if isinstance(value, (int, float)):
        return float(value)
    
    value_str = str(value).strip()
    
    # $ belgisini olib tashlash
    value_str = value_str.replace('$', '')
    
    # Vergullarni olib tashlash
    value_str = value_str.replace(',', '')
    
    # Bo'sh joylarni olib tashlash
    value_str = value_str.strip()
    
    # Raqamga o'tkazish
    try:
        return float(value_str)
    except:
        return np.nan


def standardize_loan_type(value):
    """Qarz turini standartlashtirish"""
    if pd.isna(value):
        return value
    
    value_str = str(value).strip()
    
    if 'personal' in value_str.lower():
        return 'Personal'
    elif 'mortgage' in value_str.lower():
        return 'Mortgage'
    elif 'credit' in value_str.lower() and 'card' in value_str.lower():
        return 'CreditCard'
    elif 'auto' in value_str.lower():
        return 'Auto'
    else:
        return value_str

def standardize_education(value):
    """Ta'lim darajasini standartlashtirish"""
    if pd.isna(value):
        return value
    
    value_str = str(value).strip()
    
    # Case standartlashtirish
    if 'graduate' in value_str.lower():
        return 'Graduate'
    elif 'bachelor' in value_str.lower():
        return 'Bachelor'
    elif 'high school' in value_str.lower() or 'highschool' in value_str.lower():
        return 'High School'
    elif 'some college' in value_str.lower() or 'somecollege' in value_str.lower():
        return 'Some College'
    elif 'master' in value_str.lower():
        return 'Master'
    elif 'doctorate' in value_str.lower() or 'phd' in value_str.lower():
        return 'Doctorate'
    else:
        return value_str

def standardize_marital_status(value):
    """Oilaviy holatni standartlashtirish"""
    if pd.isna(value):
        return value
    
    value_str = str(value).strip()
    
    if 'married' in value_str.lower():
        return 'Married'
    elif 'single' in value_str.lower():
        return 'Single'
    elif 'divorced' in value_str.lower():
        return 'Divorced'
    elif 'widowed' in value_str.lower():
        return 'Widowed'
    else:
        return value_str




🧹 PHASE 2: TOZALASH FUNKSIYALARI
✅ Tozalash funksiyalari yaratildi!


In [ ]:
# Datasetlarni tozalash
cleaned_datasets = {}

# APPLICATION_METADATA
df_app = datasets['application_metadata'].copy()
cleaned_datasets['application_metadata'] = df_app

# DEMOGRAPHICS
df_demo = datasets['demographics'].copy()
df_demo['annual_income'] = df_demo['annual_income'].apply(clean_currency)
df_demo['education'] = df_demo['education'].apply(standardize_education)
df_demo['marital_status'] = df_demo['marital_status'].apply(standardize_marital_status)
if df_demo['employment_length'].isnull().sum() > 0:
    df_demo['employment_length'] = df_demo['employment_length'].fillna(df_demo['employment_length'].median())
cleaned_datasets['demographics'] = df_demo

# CREDIT_HISTORY
df_credit = datasets['credit_history'].copy()
if df_credit['num_delinquencies_2yrs'].isnull().sum() > 0:
    df_credit['num_delinquencies_2yrs'] = df_credit['num_delinquencies_2yrs'].fillna(0)
cleaned_datasets['credit_history'] = df_credit

# LOAN_DETAILS
df_loan = datasets['loan_details'].copy()
df_loan['loan_amount'] = df_loan['loan_amount'].apply(clean_currency)
df_loan['loan_type'] = df_loan['loan_type'].apply(standardize_loan_type)
cleaned_datasets['loan_details'] = df_loan

# FINANCIAL_RATIOS
df_fin = datasets['financial_ratios'].copy()
currency_columns = [
    'monthly_income', 'existing_monthly_debt', 'monthly_payment',
    'revolving_balance', 'credit_usage_amount', 'available_credit',
    'total_monthly_debt_payment', 'total_debt_amount', 'monthly_free_cash_flow'
]
for col in currency_columns:
    if col in df_fin.columns:
        df_fin[col] = df_fin[col].apply(clean_currency)
if 'revolving_balance' in df_fin.columns and df_fin['revolving_balance'].isnull().sum() > 0:
    df_fin['revolving_balance'] = df_fin['revolving_balance'].fillna(0)
cleaned_datasets['financial_ratios'] = df_fin

# GEOGRAPHIC_DATA
df_geo = datasets['geographic_data'].copy()
cleaned_datasets['geographic_data'] = df_geo


🧹 PHASE 3: DATASETLARNI TOZALASH

1️⃣ APPLICATION_METADATA tozalash...


   ✅ Tozalandi: (89999, 14)

2️⃣ DEMOGRAPHICS tozalash...
   ⚠️ employment_length NaN to'ldirildi (median: 5.20)
   ✅ Tozalandi: (89999, 8)

3️⃣ CREDIT_HISTORY tozalash...
   ⚠️ num_delinquencies_2yrs NaN to'ldirildi (0)
   ✅ Tozalandi: (89999, 12)

4️⃣ LOAN_DETAILS tozalash...
   ✅ Tozalandi: (89999, 10)

5️⃣ FINANCIAL_RATIOS tozalash...
   ✅ monthly_income tozalandi
   ✅ existing_monthly_debt tozalandi
   ✅ monthly_payment tozalandi
   ✅ revolving_balance tozalandi
   ✅ credit_usage_amount tozalandi
   ✅ available_credit tozalandi
   ✅ total_monthly_debt_payment tozalandi
   ✅ total_debt_amount tozalandi
   ✅ monthly_free_cash_flow tozalandi
   ⚠️ revolving_balance NaN to'ldirildi (0)
   ✅ Tozalandi: (89999, 16)

6️⃣ GEOGRAPHIC_DATA tozalash...
   ✅ Tozalandi: (89999, 8)

✅ Barcha datasetlar tozalandi!


In [ ]:
# Datasetlarni birlashtirish (JOIN)
df_final = cleaned_datasets['application_metadata'].copy()

df_final = df_final.merge(
    cleaned_datasets['demographics'],
    left_on='customer_ref',
    right_on='cust_id',
    how='left',
    suffixes=('', '_demo')
)

df_final = df_final.merge(
    cleaned_datasets['credit_history'],
    left_on='customer_ref',
    right_on='customer_number',
    how='left',
    suffixes=('', '_credit')
)

df_final = df_final.merge(
    cleaned_datasets['loan_details'],
    left_on='customer_ref',
    right_on='customer_id',
    how='left',
    suffixes=('', '_loan')
)

df_final = df_final.merge(
    cleaned_datasets['financial_ratios'],
    left_on='customer_ref',
    right_on='cust_num',
    how='left',
    suffixes=('', '_financial')
)

df_final = df_final.merge(
    cleaned_datasets['geographic_data'],
    left_on='customer_ref',
    right_on='id',
    how='left',
    suffixes=('', '_geo')
)


🔗 PHASE 4: DATASETLARNI BIRLASHTIRISH

📊 Asosiy dataset: application_metadata
   O'lchami: (89999, 14)

1️⃣ Demographics join...
   ✅ Join qilindi: (89999, 22)

2️⃣ Credit History join...
   ✅ Join qilindi: (89999, 34)

3️⃣ Loan Details join...
   ✅ Join qilindi: (89999, 44)

4️⃣ Financial Ratios join...
   ✅ Join qilindi: (89999, 60)

5️⃣ Geographic Data join...
   ✅ Join qilindi: (89999, 68)

✅ Barcha datasetlar birlashtirildi!
   Final o'lchami: (89999, 68)
   Qatorlar: 89,999
   Ustunlar: 68


In [ ]:
# Takrorlangan va foydasiz ustunlarni o'chirish
columns_to_drop = [
    'cust_id',
    'customer_number',
    'customer_id',
    'cust_num',
    'id',
    'recent_inquiry_count',
    'random_noise_1'
]

existing_drop_cols = [col for col in columns_to_drop if col in df_final.columns]
if existing_drop_cols:
    df_final = df_final.drop(columns=existing_drop_cols)


🗑️ PHASE 5: TAKRORLANGAN USTUNLARNI O'CHIRISH

🗑️ O'chiriladigan ustunlar (7 ta):
   - cust_id
   - customer_number
   - customer_id
   - cust_num
   - id
   - recent_inquiry_count
   - random_noise_1

✅ Ustunlar o'chirildi
   Yangi o'lchami: (89999, 61)

📋 Final dataset ustunlari (61 ta):
     1. customer_ref                             | int64           | NaN:      0 ( 0.00%)
     2. application_id                           | int64           | NaN:      0 ( 0.00%)
     3. application_hour                         | int64           | NaN:      0 ( 0.00%)
     4. application_day_of_week                  | int64           | NaN:      0 ( 0.00%)
     5. account_open_year                        | int64           | NaN:      0 ( 0.00%)
     6. preferred_contact                        | object          | NaN:      0 ( 0.00%)
     7. referral_code                            | object          | NaN:      0 ( 0.00%)
     8. account_status_code                      | object          | NaN:      

In [ ]:
# Final validatsiya
output_file = Path('final_dataset.csv')
df_final.to_csv(output_file, index=False)


✅ PHASE 6: FINAL VALIDATSIYA VA SAQLASH

📊 Final dataset ma'lumotlari:
   O'lchami: (89999, 61)
   Qatorlar: 89,999
   Ustunlar: 61

🎯 Target ustun (default) taqsimoti:
default
0    85405
1     4594
Name: count, dtype: int64

   Foizlar:
default
0    94.9
1     5.1
Name: proportion, dtype: float64

   ⚠️ Eslatma: 0 = XAVFLI (to'lay olmaydi), 1 = XAVFSIZ (to'lay oladi)

✅ Yo'qolgan qiymatlar yo'q!

🔍 Birinchi 5 qator:


,customer_ref,application_id,application_hour,application_day_of_week,account_open_year,preferred_contact,referral_code,account_status_code,num_login_sessions,num_customer_service_calls,has_mobile_app,paperless_billing,default,age,annual_income,employment_length,employment_type,education,marital_status,num_dependents,credit_score,num_credit_accounts,oldest_credit_line_age,oldest_account_age_months,total_credit_limit,num_delinquencies_2yrs,num_inquiries_6mo,num_public_records,num_collections,account_diversity_index,loan_type,loan_amount,loan_term,interest_rate,loan_purpose,loan_to_value_ratio,origination_channel,loan_officer_id,marketing_campaign,monthly_income,existing_monthly_debt,monthly_payment,debt_to_income_ratio,debt_service_ratio,payment_to_income_ratio,credit_utilization,revolving_balance,credit_usage_amount,available_credit,total_monthly_debt_payment,annual_debt_payment,loan_to_annual_income,total_debt_amount,monthly_free_cash_flow,cost_of_living_index,housing_price_index,previous_zip_code,regional_median_income,regional_median_rent,regional_unemployment_rate,state
0,10000,620515,5,6,2013,Mail,REF0000,ACT-2,13,2,1,1,0,41,61800.0,2.2,Full-time,Graduate,Married,2,696,14,22.8,273.6,169100.0,0.0,2,1,0,0.499,Personal,17700.0,36,12.50,Debt Consolidation,0.000,Direct Mail,1045,W,5150.00,738.64,592.13,0.258,0.258402,0.115,0.841,142213.1,142213.1,26886.9,1330.77,15969.24,0.286408,159913.1,3819.230000,73.0,91.0,451,56000,1380.0,4.8,OH
1,10001,624978,4,2,2015,Phone,REF0000,ACT-3,6,1,1,1,1,38,28600.0,7.0,FULL_TIME,High School,Married,0,659,13,3.5,42.0,78200.0,0.0,6,0,0,0.298,Mortgage,114000.0,180,6.83,Refinance,0.774,Branch,1011,B,2383.33,392.21,1013.86,0.590,0.589959,0.425,0.971,75932.2,75932.2,2267.8,1406.07,16872.84,3.986014,189932.2,977.260000,87.0,92.0,537,61000,1510.0,4.4,PA
2,10002,564658,10,3,2020,Phone,REF0000,ACT-3,1,2,1,0,0,18,20700.0,0.8,FULL_TIME,Bachelor,Single,0,662,3,0.0,0.0,41400.0,0.0,2,0,0,0.174,Personal,9300.0,36,13.99,Major Purchase,0.000,Online,1084,K,1725.00,204.07,317.81,0.303,0.302539,0.184,0.539,22314.6,22314.6,19085.4,521.88,6262.56,0.449275,31614.6,1203.120000,103.0,125.0,679,74000,1920.0,3.9,VA
3,10003,621493,7,5,2010,Email,REF0000,A01,4,1,1,1,0,27,31400.0,4.8,Full Time,Bachelor,Single,0,676,8,9.0,108.0,60000.0,0.0,1,0,0,0.263,Personal,8700.0,48,13.26,Medical,0.000,Online,1048,A,2616.67,288.71,234.52,0.200,0.199961,0.090,0.147,8820.0,8820.0,51180.0,523.23,6278.76,0.277070,17520.0,2093.436667,121.0,158.0,719,75000,1690.0,5.8,CA
4,10004,637785,1,2,2020,Mail,REF0000,ACT-3,6,2,1,0,0,26,24600.0,5.2,Fulltime,High School,Single,0,678,7,8.0,96.0,49700.0,0.0,1,0,0,0.298,Personal,7200.0,24,10.77,Debt Consolidation,0.000,Branch,1055,S,2050.00,248.77,334.81,0.285,0.284673,0.163,0.488,24253.6,24253.6,25446.4,583.58,7002.96,0.292683,31453.6,1466.420000,127.0,152.0,933,78000,1700.0,5.8,WA


In [ ]:
# Qo'shimcha tozalash funksiyalari
def clean_employment_type(value):
    if pd.isna(value):
        return value
    value_str = str(value).strip().lower()
    if value_str in ['full-time', 'full time', 'fulltime', 'ft', 'full_time']:
        return 'Full-time'
    elif value_str in ['part-time', 'part time', 'parttime', 'pt', 'part_time']:
        return 'Part-time'
    elif value_str in ['self-employed', 'self employed', 'selfemp', 'self emp', 'self-emp', 'self_employed']:
        return 'Self-employed'
    elif value_str in ['contractor', 'contract']:
        return 'Contractor'
    return str(value).strip()

def clean_education(value):
    if pd.isna(value):
        return value
    value_lower = str(value).lower()
    if value_lower in ['some college', 'high school', 'high_school']:
        return 'Low Education'
    elif value_lower in ['bachelor']:
        return 'Medium Education'
    elif value_lower in ['graduate', 'advanced']:
        return 'High Education'
    return value

def clean_marital_status(value):
    if pd.isna(value):
        return value
    return 'Married' if 'married' in str(value).lower() else 'Not Married'

def clean_loan_type(value):
    if pd.isna(value):
        return value
    value_lower = str(value).lower()
    if 'cc' in value_lower or 'credit' in value_lower and 'card' in value_lower:
        return 'CreditCard'
    elif 'mortgage' in value_lower:
        return 'Home Loan'
    return value

def clear_origination_channel(value):
    if pd.isna(value):
        return value
    value_lower = str(value).lower()
    if value_lower in ['online', 'direct mail']:
        return 'Digital'
    elif value_lower in ['branch', 'broker']:
        return 'Physical'
    return value

def convert_marital_status(value):
    return 1 if value == 'Married' else 0

def convert_origination_channel(value):
    return 1 if value == 'Digital' else 0

# Tozalash funksiyalarini qo'llash
df_final['employment_type'] = df_final['employment_type'].apply(clean_employment_type)
df_final['education'] = df_final['education'].apply(clean_education)
df_final['marital_status'] = df_final['marital_status'].apply(clean_marital_status)
df_final['loan_type'] = df_final['loan_type'].apply(clean_loan_type)
df_final['origination_channel'] = df_final['origination_channel'].apply(clear_origination_channel)
df_final['marital_status'] = df_final['marital_status'].apply(convert_marital_status)
df_final['origination_channel'] = df_final['origination_channel'].apply(convert_origination_channel)

# Final datasetni saqlash
output_file = Path('final_dataset.csv')
df_final.to_csv(output_file, index=False)

💾 FINAL DATASETNI SAQLASH

✅ Final dataset saqlandi: final_dataset.csv
   O'lchami: (89999, 61)
   Qatorlar: 89,999
   Ustunlar: 61
   Fayl hajmi: 31.87 MB

🎉 Barcha ishlar muvaffaqiyatli yakunlandi!

📋 Keyingi qadamlar:
   1. Final datasetni ML model uchun tayyorlash
   2. Feature engineering (yangi ustunlar yaratish)
   3. Model training (Default Prediction va Loan Amount Prediction)


In [ ]:
# Foydasiz ustunlarni o'chirish va for_model papkasiga saqlash
columns_to_drop = ['state', 'marketing_campaign']
existing_drop_cols = [col for col in columns_to_drop if col in df_final.columns]

if existing_drop_cols:
    df_final_cleaned = df_final.drop(columns=existing_drop_cols)
else:
    df_final_cleaned = df_final.copy()

# for_model papkasiga saqlash
for_model_folder = Path('../for_model')
for_model_folder.mkdir(exist_ok=True)
output_file = for_model_folder / 'dataset_for_models.csv'
df_final_cleaned.to_csv(output_file, index=False)


🗑️ FOYDASIZ USTUNLARNI O'CHIRISH

🗑️ O'chiriladigan ustunlar (2 ta):
   - state
   - marketing_campaign

✅ Ustunlar o'chirildi
   Eski o'lchami: (89999, 61)
   Yangi o'lchami: (89999, 59)

📊 Tozalangan dataset:
   Qatorlar: 89,999
   Ustunlar: 59
